In [13]:
import numpy as np
# from pymoo.core.problem import Problem
from scipy.interpolate import griddata
import pandas as pd

from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem import ElementwiseProblem
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.optimize import minimize
from pymoo.termination.default import DefaultSingleObjectiveTermination

In [14]:
jpeg_cha = pd.read_csv("../../stable_tests/measurements/jpeg_snr_cha/characteristic.csv")

jpeg_cha = jpeg_cha[jpeg_cha["sparsity"]>0]
jpeg_cha = jpeg_cha[jpeg_cha["datasize_est"]>0]
tensor_size = 128*26*26 *4
jpeg_cha["ratio"] = tensor_size/jpeg_cha["datasize_est"]

In [15]:
pruning = [0.05, 0.1, 0.15, 0.2, 0.25,0.3,0.35]
quality = [60,70,80,90,100]

sample_points=[]
cmp_sample_values = []
snr_sample_values = []

cha_df_group =jpeg_cha.groupby("pruning_thresh")
for p in pruning:
    cha_df = cha_df_group.get_group(p)
    cha_quality_df = cha_df.groupby("quality")
    for q in quality:
        cha_plot_df= cha_quality_df.get_group(q)
        sample_points.append([q,p])
        cmp_sample_values.append(cha_plot_df["ratio"].mean())
        snr_sample_values.append(cha_plot_df["reconstruct_snr"].mean())

sample_points = np.array(sample_points)
cmp_sample_values = np.array(cmp_sample_values)
snr_sample_values = np.array(snr_sample_values)

In [16]:
# Interpolate the constraint function
def interpolated_constraint_cmp(xy):
    return griddata(sample_points, cmp_sample_values, (xy[0], xy[1]/100), method='cubic')

def interpolated_constraint_snr(xy):
    return griddata(sample_points, snr_sample_values, (xy[0], xy[1]/100), method='cubic')


class CustomProblem(ElementwiseProblem):

    def __init__(self,snr,cmp):
        self.snr = snr
        self.cmp = cmp
        super().__init__(n_var=2, n_obj=1, n_ieq_constr=2, xl=[60,5], xu=[100,35],vtype=int)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = x[1]/35-x[0]/100

        constraint_1 = self.cmp-interpolated_constraint_cmp(x)
        constraint_2 = self.snr-interpolated_constraint_snr(x)

        out["G"] = np.array([
            constraint_1,constraint_2
        ])
        penalty = 1000  # Large penalty factor
        if constraint_1 > 0:  # If constraint is violated
            out["F"] += penalty * constraint_1
        if constraint_1 > 0:  # If constraint is violated
            out["F"] += penalty * constraint_2

In [17]:
# Initialize the problem
problem = CustomProblem(snr=25,cmp=21)

# Select the algorithm (Genetic Algorithm in this case)
algorithm = GA(pop_size=25,
            sampling=IntegerRandomSampling(),
            crossover=SBX(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
            mutation=PM(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
            eliminate_duplicates=True,
            )

termination = DefaultSingleObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=1e-6,
    period=20,
    n_max_gen=20,
    n_max_evals=10000
)


# Perform the optimization
result = minimize(problem, algorithm, termination=termination,verbose=True)

# # Get and display the results
# optimal_x, optimal_y = result.X
# optimal_objective_value = -result.F[0]

print("Best solution found: \nX = %s\nF = %s" % (result.X, result.F))

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       25 |  6.9087175408 |  2.002044E+01 |             - |             -
     2 |       50 |  6.9087175408 |  1.376814E+01 |             - |             -
     3 |       75 |  3.2104995422 |  8.7240144005 |             - |             -
     4 |      100 |  2.8278844159 |  6.1061812009 |             - |             -
     5 |      125 |  1.4935417304 |  4.2049794834 |             - |             -
     6 |      150 |  1.4935417304 |  3.6917733920 |             - |             -
     7 |      175 |  1.4935417304 |  3.4840817067 |             - |             -
     8 |      200 |  1.4935417304 |  3.1697345258 |             - |             -
     9 |      225 |  1.4935417304 |  3.1345651492 |             - |             -
    10 |      250 |  1.4935417304 |  3.1192656026 |             - |             -
    11 |      275 |  1.4935417304 |  3.0909838815 |             - |             -
    12 |      30

In [18]:
result.X[1]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
interpolated_constraint_cmp(result.X)

array(8.8830775)

In [ ]:
interpolated_constraint_snr(result.X)

array(35.17396099)